### COMP90051

## Project2

# Process the data

In [15]:
import copy
from sklearn.decomposition import PCA
import numpy as np
from tensorflow import keras
import json



data_file = open('train.json', 'r')
train_data = json.load(data_file)

In [2]:
prolific = []
titles = []
abstracts = []
titles_and_abstracts = []
title_words_by_author= np.zeros((100,5000))

for i in range(len(train_data)):
  auts = np.array(train_data[i]['authors'])
  auts = auts[auts < 100]
  cau = auts[auts >= 100]
  authors = np.zeros(100)
  if len(auts) > 0:
    authors[auts] = 1
  prolific.append(authors)

  title = train_data[i]["title"]
  titles.append(title)           # there are 3444 words appear in the title
  abstracts.append(train_data[i]["abstract"])
  titles_and_abstracts.append(title + train_data[i]["abstract"])
  for j in title:
    for m in auts:
      title_words_by_author[m,j] = title_words_by_author[m,j] + 1
prolific = np.array(prolific)


venues = []
for i in train_data :
  venue = np.zeros(466)
  vn = i["venue"]
  if vn == "":
    venue[-1] = 1
  else:
    venue[vn] = 1
  venues.append(venue)
venues = np.array(venues)

In [3]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=8)
svd.fit(venues)
venues = svd.transform(venues)


Get the 25 most frequently used words in the title by each author

In [4]:
top25words = []
for i in range(title_words_by_author.shape[0]):
  words = title_words_by_author[i,:].tolist()
  max_times = []
  for _ in range(25):
    idx = words.index(max(words))
    words[idx] = 0
    max_times.append(idx)
  top25words.append(max_times)
top25words = np.array(top25words)

Assign different weight groups based on how words are used by each author


In [5]:
unique, counts = np.unique(top25words, return_counts=True)
common_words = unique[np.where( counts > 50)]
field_words = unique[np.where( (counts <= 50) & (counts >= 10))]
personal_word = unique[np.where( counts < 10)]

In [6]:
import gensim

vsize = 64
title_word2vec = gensim.models.Word2Vec(sentences = titles, vector_size = vsize)  # min_count = 5,


# Weighted average for title vectors
titlevec = []
com_we = -0.8
fie_we = 0.2
per_we = 1

In [7]:
# Weighted average
for tit in titles:
  vec = np.zeros(vsize)
  l = len(tit)
  for j in tit:
    if j in title_word2vec.wv.key_to_index:
      if j in common_words:
        vec += title_word2vec.wv[j] * (1 + com_we)
        l += com_we
      elif j in field_words:
        vec += title_word2vec.wv[j] * (1 + fie_we)
        l += fie_we
      else:
        vec += title_word2vec.wv[j] * (1 + per_we)
        l += per_we
  vec = vec/l
  titlevec.append(vec)
titlevec = np.array(titlevec)

In [8]:
# Doc2Vec 
for i in range(len(abstracts)):
  ab = []
  for w in abstracts[i]:
    ab.append(str(w))
  abstracts[i] = ab

documents = [gensim.models.doc2vec.TaggedDocument(abstracts[i], str([i])) for i in range(len(abstracts))]
model = gensim.models.doc2vec.Doc2Vec(documents, vector_size=64 , min_count = 1)

abst_vec = []
for abst in abstracts:
  vect = model.infer_vector(abst)
  abst_vec.append(vect)

abst_vec = np.array(abst_vec)

In [9]:
train_feature = np.concatenate((titlevec,abst_vec),axis= 1)
print(train_feature.shape)
train_feature= np.concatenate((train_feature,venues),axis= 1)
print(train_feature.shape)

(25793, 128)
(25793, 136)


In [10]:
np.save("feature_train.npy",train_feature)
np.save("labels.npy",prolific)

In [11]:
data_file = open('test.json', 'r')
test_data = json.load(data_file)

abst_vec = []
titlevec = []
for i in range(len(test_data)):
  abst = test_data[i]["abstract"]
  ab = []
  for w in abst:
    ab.append(str(w))
  vect = model.infer_vector(ab)
  abst_vec.append(vect)

  tit = test_data[i]["title"]
  vec = np.zeros(vsize)
  l = len(tit)
  for j in tit:
    if j in title_word2vec.wv.key_to_index:
      if j in common_words:
        vec += title_word2vec.wv[j] * (1 + com_we)
        l += com_we
      elif j in field_words:
        vec += title_word2vec.wv[j] * (1 + fie_we)
        l += fie_we
      else:
        vec += title_word2vec.wv[j] * (1 + per_we)
        l += per_we
  vec = vec/l
  titlevec.append(vec)
titlevec = np.array(titlevec)
abst_vec = np.array(abst_vec)

In [12]:
venues = []
for i in test_data :
  venue = np.zeros(466)
  vn = i["venue"]
  if vn == "":
    venue[-1] = 1
  else:
    venue[vn] = 1
  venues.append(venue)
venues = np.array(venues)
venues = svd.transform(venues)

In [14]:
test_feature = np.concatenate((titlevec,abst_vec),axis= 1)
test_feature= np.concatenate((test_feature,venues),axis= 1)
print(test_feature.shape)

(800, 136)


In [ ]:
np.save("test_feature.npy",test_feature)

# 

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# the path of the data
path = "/content/drive/MyDrive/COMP90051/data/"

In [ ]:
%cd {path}
!ls

In [ ]:
x= np.load('feature_train.npy')
y= np.load('labels.npy')

print(x.shape,y.shape)

# co author

In [ ]:
%cd /content/drive/MyDrive/COMP90051/
!ls

In [ ]:
import json
from sklearn.decomposition import PCA

data_file = open('train.json', 'r')
train_data = json.load(data_file)


In [ ]:
#
coauthor = []
for d in train_data:
  au = np.array(d['authors'])
  
  coa = au[au >= 100]
  co = np.zeros(21146)
  for c in coa:
    co[c-100] = 1
  coauthor.append(co)
coauthor = np.array(coauthor)



In [ ]:
coauthor.shape

(25793, 21146)

In [ ]:
# from sklearn.decomposition import TruncatedSVD
# 
# svd_c = TruncatedSVD(n_components=2048)
# svd_c.fit(coauthor)
# coauthor = svd_c.transform(coauthor)

In [ ]:
x = np.concatenate((x,coauthor),axis= 1)
print(x.shape)

(25793, 21282)


In [ ]:
data_file = open('test.json', 'r')
test_data = json.load(data_file)

In [ ]:

coauthor = []
for d in test_data:
  coa = np.array(d['coauthors'])
  co = np.zeros(21146)
  for c in coa:
    co[c-100] = 1
  coauthor.append(co)
coauthor = np.array(coauthor)
coauthor.shape

(800, 21146)

In [ ]:
#coauthor = svd_c.transform(coauthor)
coauthor.shape

(800, 1024)

In [ ]:
%cd {path}
np.save("feature_train.npy",x)

test_feature = np.load('test_feature.npy')

test_feature= np.concatenate((test_feature,coauthor),axis= 1)
print(test_feature.shape)
np.save("test_feature.npy",test_feature)

/content/drive/.shortcut-targets-by-id/1zMevzR3uxycPOSTvIoERp_FudjvEl5t5/COMP90051/data/coauthor_21146
(800, 21282)


# predict

In [6]:
test_feature = np.load("test_feature.npy")

In [7]:
print(x.shape,test_feature.shape,y.shape)

(25793, 21282) (800, 21282) (25793, 101)


In [8]:
scaler = StandardScaler()
x =scaler.fit_transform(x)

In [9]:
class_weight = {}
for i in range(100):
    class_weight[i] = 1
class_weight[100] = 0.25

In [ ]:
x_train,x_test, y_train,y_test =train_test_split(x,y,test_size=0.2, random_state=1)

In [10]:
x_train = x
y_train = y

In [11]:
model = keras.Sequential(
    [
        layers.Input((x_train.shape[1],)),
        layers.Dense(1048, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(y_train.shape[1],activation='sigmoid'),
    ],
)

In [12]:
#model.compile(loss= tf.keras.losses.CategoricalCrossentropy() )
model.compile(loss='binary_crossentropy')
model.fit(x_train,y_train,epochs=100,batch_size=512,class_weight = class_weight)

Epoch 1/100
51/51 [==============================] - 5s 42ms/step - loss: 0.0388
Epoch 2/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0178
Epoch 3/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0107
Epoch 4/100
51/51 [==============================] - 2s 43ms/step - loss: 0.0071
Epoch 5/100
51/51 [==============================] - 3s 58ms/step - loss: 0.0054
Epoch 6/100
51/51 [==============================] - 3s 54ms/step - loss: 0.0045
Epoch 7/100
51/51 [==============================] - 3s 62ms/step - loss: 0.0038
Epoch 8/100
51/51 [==============================] - 2s 41ms/step - loss: 0.0034
Epoch 9/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0030
Epoch 10/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0029
Epoch 11/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0025
Epoch 12/100
51/51 [==============================] - 2s 39ms/step - loss: 0.0025
Epoch 13/100
51/51 [=====

In [ ]:
pred =model.predict(x)
pred=np.where(pred>=0.3,1,0)
f1_score(y_true=y,y_pred=pred,average='samples')

807/807 [==============================] - 2s 2ms/step


0.9122533033933532

In [ ]:
test_pred=model.predict(x_test)
test_pred= np.where(test_pred>=0.3,1,0)
f1_score(y_true=y_test,y_pred=test_pred,average='samples')

162/162 [==============================] - 0s 2ms/step


0.5626022684849906

In [14]:
test_data = scaler.transform(test_feature)
predd =model.predict(test_data)
idxx = [np.where(p>=0.3) for p in predd]

25/25 [==============================] - 0s 3ms/step


In [15]:
df= pd.DataFrame(idxx,columns=['predict'])
df.to_csv('predict.csv')

In [ ]:
test_feature = np.load("/content/drive/MyDrive/COMP90051/data/test_feature.npy")
scaler = StandardScaler()
x = scaler.fit_transform(x)
test_feature = scaler.transform(test_feature)

In [ ]:
import xgboost as xgb
clf = xgb.XGBClassifier(max_depth=3)
clf.fit(x, authors[0])
predictions = clf.predict(x)

In [ ]:
# random data to test the xgboost


import xgboost as xgb
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score


X, y = make_multilabel_classification(n_samples=3000, n_features=45, n_classes=20, n_labels=1,
                                      allow_unlabeled=False, random_state=42)

